In [1]:
import pennylane as qml
from pennylane import numpy as np
import QEMZMSEPC as qem

In [2]:
n_qubits = 2
dev = qml.device('default.mixed', wires=n_qubits)
operations = ['RX', 'RY', 'RZ', 'RX', 'RY', 'RZ', 'CNOT', 'CNOT',
              'RX', 'RY', 'RZ', 'RX', 'RY', 'RZ', 'CNOT', 'CNOT']
paras = [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 1], [1, 0],
         [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 1], [1, 0]]
nqubitspaulimatrices = qem.NqubitsPauliMatrices(n_qubits)
pauli_set_of_n_qubits = nqubitspaulimatrices.get_pauli_matrices_of_n_qubits()
nqubitschannel = qem.NqubitsChannel(n_qubits, pauli_set_of_n_qubits)
nqubitspaulichannel = nqubitschannel.nqubitsrandompaulichannel(p_identity=0.8)
nqubitsdepolarizingchannel = nqubitschannel.nqubitsdepolarizingchannel(0.9)
qemzmsepc = qem.QEMZMSEPC(n_qubits)
_, p_t = qemzmsepc.qemzmsepc(operations=operations, paras=paras, dev=dev, p=0.9,
                             kraus_matrices_of_a_pauli_channel=nqubitspaulichannel)
print(p_t)

0.8750582834828504


In [3]:
@qml.qnode(qml.device('default.mixed', wires=n_qubits))
def train_cir_with_qem(parameters):
    qml.StronglyEntanglingLayers(weights=parameters, wires=range(2))
    qml.QubitChannel(nqubitsdepolarizingchannel, wires=[0, 1])
    qml.QubitChannel(nqubitspaulichannel, wires=[0, 1])
    return qml.expval(qml.PauliZ(0)@qml.PauliZ(1))


shape = qml.StronglyEntanglingLayers.shape(n_layers=2, n_wires=2)
weights = np.random.random(size=shape)


def cost(x):
    return ((train_cir_with_qem(x)/p_t)-(-1))**2

opt = qml.GradientDescentOptimizer(stepsize=0.1)

lst = []
# set the number of steps
steps = 100
# set the initial parameter values
params = weights

for i in range(steps):
    # update the circuit parameters
    params = opt.step(cost, params)

    if (i + 1) % 5 == 0:
        print("Cost after step {:5d}: {: .7f}".format(i + 1, cost(params)))
        lst.append(cost(params))
print("Optimized rotation angles: {}".format(params))


Cost after step     5:  0.3518357
Cost after step    10:  0.1084642
Cost after step    15:  0.0521370
Cost after step    20:  0.0311087
Cost after step    25:  0.0209547
Cost after step    30:  0.0152320
Cost after step    35:  0.0116613
Cost after step    40:  0.0092679
Cost after step    45:  0.0075765
Cost after step    50:  0.0063315
Cost after step    55:  0.0053852
Cost after step    60:  0.0046469
Cost after step    65:  0.0040584
Cost after step    70:  0.0035807
Cost after step    75:  0.0031869
Cost after step    80:  0.0028581
Cost after step    85:  0.0025802
Cost after step    90:  0.0023430
Cost after step    95:  0.0021387
Cost after step   100:  0.0019614
Optimized rotation angles: [[[ 0.69733798  1.26640733  0.3220726 ]
  [ 0.52657648  2.42215748  0.21817786]]

 [[-0.08238892  0.70625101  0.81101352]
  [ 0.23009393  0.61641137  0.5280929 ]]]


In [4]:
@qml.qnode(qml.device('default.mixed', wires=n_qubits))
def train_cir_without_qem(parameters):
    qml.StronglyEntanglingLayers(weights=parameters, wires=range(2))
    qml.QubitChannel(nqubitsdepolarizingchannel, wires=[0, 1])
    qml.QubitChannel(nqubitspaulichannel, wires=[0, 1])
    return qml.expval(qml.PauliZ(0)@qml.PauliZ(1))


shape = qml.StronglyEntanglingLayers.shape(n_layers=2, n_wires=2)
weights = np.random.random(size=shape)

def cost(x):
    return ((train_cir_without_qem(x))-(-1))**2

opt = qml.GradientDescentOptimizer(stepsize=0.1)

lst = []
# set the number of steps
steps = 100
# set the initial parameter values
params = weights

for i in range(steps):
    # update the circuit parameters
    params = opt.step(cost, params)

    if (i + 1) % 5 == 0:
        print("Cost after step {:5d}: {: .7f}".format(i + 1, cost(params)))
        lst.append(cost(params))
print("Optimized rotation angles: {}".format(params))

Cost after step     5:  3.0141937
Cost after step    10:  0.7783178
Cost after step    15:  0.1706781
Cost after step    20:  0.0764837
Cost after step    25:  0.0481006
Cost after step    30:  0.0357251
Cost after step    35:  0.0291371
Cost after step    40:  0.0251826
Cost after step    45:  0.0226162
Cost after step    50:  0.0208588
Cost after step    55:  0.0196085
Cost after step    60:  0.0186938
Cost after step    65:  0.0180104
Cost after step    70:  0.0174917
Cost after step    75:  0.0170930
Cost after step    80:  0.0167838
Cost after step    85:  0.0165421
Cost after step    90:  0.0163521
Cost after step    95:  0.0162021
Cost after step   100:  0.0160831
Optimized rotation angles: [[[ 0.75714024  1.51031633  0.02599686]
  [ 0.23630107  1.45225048 -0.28068405]]

 [[ 0.2836144   1.686108    0.66316209]
  [ 0.77238239  0.68440789  0.95067224]]]
